
# 🏛️ The Dining Philosophers Problem

## **📜 Introduction**
The **Dining Philosophers Problem** is a **classical synchronization problem** in computer science. It models **resource allocation and concurrency** in multi-threaded systems.

### **🔹 History and Importance**
- Proposed by **Edsger W. Dijkstra** in **1965** as a study on **resource sharing and deadlocks**.
- Demonstrates **issues of deadlock, starvation, and fairness** in concurrent computing.
- Used in **operating systems, multi-threading, and distributed computing**.

## **🍽️ Problem Definition**
- **Five philosophers** sit at a circular table.
- Each philosopher has **one plate** but needs **two chopsticks** to eat.
- Chopsticks are **shared between adjacent philosophers**.
- Philosophers have **two possible states**:
  - **Thinking** 🤔: Not using chopsticks.
  - **Eating** 🍜: Must acquire **both left and right chopsticks**.

## **⚠️ The Challenge**
1. **Deadlocks** occur if all philosophers pick up their left chopstick at the same time, causing an infinite wait.
2. **Starvation** happens if some philosophers never get access to chopsticks because others monopolize them.


# 🛠️ Solutions to the Dining Philosophers Problem

We will explore **three different approaches** to solving the problem using **thread synchronization techniques**:

## **🔒 1. Using Locks (Mutex-Based Solution)**
- **Each philosopher picks up one chopstick at a time using `threading.Lock()`**.
- Prevents **race conditions** but can still **lead to deadlocks**.

## **🚦 2. Using Semaphores (Avoiding Deadlocks)**
- **Uses `threading.Semaphore()` to regulate chopstick access**.
- Helps **prevent deadlocks** but may still cause **starvation**.

## **✅ 3. Using Conditional Variables (Fair Scheduling)**
- **Uses `threading.Condition()` to ensure that a philosopher waits until both chopsticks are free**.
- **Prevents deadlocks and ensures fairness**.

Each of these solutions will be explained **with code and detailed explanations** below.


## 🔒 Solution 1: Using Locks (Mutex)

In [ ]:
import time 
import random
import threading 

In [ ]:
class DiningPhilosophers_w_Locks:
    def __init__(self, total_phils_no=5, meal_size=7):  
        # total_phils_no: the number of philosophers on the table
        
        # meal_size: indicator of the size of the meal that philosophers have on their plate, i.e. how many times they need to 'eat' in order for them to finish their plate
        self.meals = [meal_size for _ in range(total_phils_no)]  # array w/ the "size" of the meal left on each philosopher's plate (i.e. how many times they need to 'eat' to finish)
        
        # each chopstick can be held by only one philosopher at a given time - we will implement a lock for each, since there are as many chopsticks as there are philosophers
        self.chopsticks = [threading.Lock() for _ in range(total_phils_no)] 

        self.status = ['  T  ' for _ in range(total_phils_no)]  # at the beginning, all philosophers are thinking - no one is eating
        self.chopstick_holders = ['     ' for _ in range(total_phils_no)] # this will show how many philosophers have both chopsticks on their hands at a given time; at the beginning, no one does 

    # method used to instantiate and spawn our threads
    def philosopher(self, i):  # i: a unique ID for the philosopher considered at any given time
        while self.meals[i] > 0:  # i.e. if there need to be implemented more 'eating' jobs
            
            self.status[i] = '  T  '
            time.sleep(random.random())  # Representing the state where the philosopher is 'thinking' about deciding to eat, i.e. attempting to obtain a right-hand side chopstick.
            
            # We need to check the status of the left-hand chopstick of each philosopher, to make sure that it has not been already acquired by the philosopher to their left.
            self.status[i] = '  _  '  # intermediate situation between thinking and eating; waiting to acquire two chopsticks

            if not self.chopsticks[i].locked():
                # if not locked, then MAYBE they could be able to eat
                self.chopsticks[i].acquire()
                self.chopstick_holders[i] = ' /   '
                time.sleep(random.random())

                # Phi 0: gets the chopstick 0 and attempts to get chopstick 1
                # ...
                # Phi 4: gets the chopstick 4 and attempts to get chopstick 0
                # This is why we do not use 'i+1' but '(i+1) % 5' so that the chopstick no does not go to 5, 
                # but instead it 'circles around' 0 to 4
                j = (i+1) % 5
                if not self.chopsticks[j].locked():  # they are free to acquire their right-hand chopstick and eat! 
                    self.chopsticks[j].acquire()
                    self.chopstick_holders[i] = ' / \\ '  # representing both chopsticks being held
                    self.status[i] = '  E  '  # eating status
                    time.sleep(random.random())
                    # the philosopher eats so ...
                    self.meals[i] -= 1 
                    # now he has eaten so he should release both chopsticks
                    self.chopsticks[i].release()
                    self.chopstick_holders[i] = ' /   '
                    self.chopsticks[j].release()
                    self.chopstick_holders[i] = '     '
                
                else:
                    # since the philosopher cannot eat, they should release the other chopstick to allow for 
                    # the better flow of this process for the rest of the philosophers - prevent deadlocks!
                    self.chopsticks[i].release()
                    self.chopstick_holders[i] = '     '

In [2]:
def main_lock():
    n = 5 
    m = 9
    dining_philosophers_w_locks = DiningPhilosophers_w_Locks(n, m)
    philosophers = [] 
    for _ in range(n):
        philosopher = threading.Thread(target=dining_philosophers_w_locks.philosopher, args=[_])  # the input is the id of each philosopher considered
        philosopher.start()
        philosophers.append(philosopher)

    ### this part could be skipped - we're using it for debugging and presentational purposes ###
    while sum(dining_philosophers_w_locks.meals) > 0:
        print("=" * (n*5))
        print("".join(map(str, dining_philosophers_w_locks.status)), 
                " : ", 
                str(dining_philosophers_w_locks.status.count('  E  '))
            )
        print("". join(map(str, dining_philosophers_w_locks.chopstick_holders)))
        print(dining_philosophers_w_locks.meals)
        time.sleep(0.1)
    ### this part could be skipped - we're using it for debugging and presentational purposes ###

    for _ in range(n):
        philosopher.join()

In [ ]:
# Still though, 
# Scenario Leading to Deadlock:
# All philosophers simultaneously lock their left chopstick.
# Each philosopher then tries to lock their right chopstick, which is already held by their neighbor.
# Result: A circular chain of waiting where no philosopher can proceed.


In [3]:
main_lock()

  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  T    T    _    T    _    :  0
           /         /   
[9, 9, 9, 9, 9]
  T    T    E    T    _    :  1
           / \       /   
[9, 9, 9, 9, 9]
  T    T    E    T    _    :  1
           / \       /   
[9, 9, 9, 9, 9]
  _    T    E    T    _    :  1
 /         / \       /   
[9, 9, 9, 9, 9]
  _    T    E    T    _    :  1
 /         / \       /   
[9, 9, 9, 9, 9]
  E    T    T    T    _    :  1
 / \                 /   
[9, 9, 8, 9, 9]
  E    T    T    _    _    :  1
 / \            /    /   
[9, 9, 8, 9, 9]
  E    T    T    T    _    :  1
 / \                 /   
[9, 9, 8, 9, 9]
  E    T    _    _    T    :  1
 / \       /    /        
[9, 9, 8, 9, 9]
  E    T    _    _    T    :  1
 / \       /    /        
[9, 9, 8, 9, 9]
  E    T    _    _    T    :  1
 / \       /    /        
[9, 9, 8, 9, 9]
  E    T    _    _    _    :  1
 / \  

## 🚦 Solution 2: Using Semaphores

In [4]:
class DiningPhilosophers_w_Semaphores:
    def __init__(self, total_phils_no=5, meal_size=7):  
        self.meals = [meal_size for _ in range(total_phils_no)]  # array w/ the meal on each philosopher's plate
        # the difference with the previous class lies on how the resource is handled - 
        # the equivalent of a lock is a semaphore with value=1 (only 1 philosopher can use 
        # the chopstick at a given time). So just changing this particular part of the 
        # chopstick creation will make the two implementations equivalent 
        self.chopsticks = [threading.Semaphore(value=1) for _ in range(total_phils_no)] # since there are as many chopsticks as there are philosophers

        self.status = ['  T  ' for _ in range(total_phils_no)]  # at the beginning, all philosophers are thinking - none is eating
        self.chopstick_holders = ['     ' for _ in range(total_phils_no)] # how many have both chopsticks on their hands at a given time

    # method used to instantiate and spawn our threads
    def philosopher(self, i):  # i: a unique ID for the philosopher considered at any given time
        while self.meals[i] > 0:  # i.e. if there need to be implemented more 'eating' processes
            
            self.status[i] = '  T  '
            time.sleep(random.random())  # representing the state where the philosopher is 'thinking'
            # about deciding to eat, i.e. attempting to obtain a right-hand side chopstick
            # we need to check the status of the left-hand chopstick of each philosopher, 
            # to make sure that it has not been already acquired by their left-side philosopher
            self.status[i] = '  _  '  # intermediate situation between thinking and eating; waiting to acquire two chopsticks


            if self.chopsticks[i].acquire(timeout=1):
                # keep trying to acquire this resource for only 1 second! After this, just 
                # just go without it (i.e. it leads us to the 'else' clause) - if we manage to acquire it
                # then this is equivalent to having acquired the lock - to the lock not being locked
                self.chopstick_holders[i] = ' /   '
                time.sleep(random.random())

                # Phi 0: gets the chopstick 0 and attempts to get chopstick 1
                # ...
                # Phi 4: gets the chopstick 4 and attempts to get chopstick 0
                # This is why we do not use 'i+1' but '(i+1) % 5' so that the chopstick no does not go to 5, 
                # but instead it 'circles around' 0 to 5
                j = (i+1) % 5
                if self.chopsticks[j].acquire(timeout=1):
                    self.chopstick_holders[i] = ' / \\ '  # representing both chopsticks being held
                    self.status[i] = '  E  '  # eating status
                    time.sleep(random.random())
                    # the philosopher eats so ...
                    self.meals[i] -= 1 
                    # now he has eaten so he should release both chopsticks
                    self.chopsticks[i].release()
                    self.chopstick_holders[i] = ' /   '
                    self.chopsticks[j].release()
                    self.chopstick_holders[i] = '     '
                
                else:
                    # since the philosopher cannot eat, they should release the other chopstick to allow for 
                    # the better flow of this process for the rest of the philosophers - prevent deadlocks!
                    self.chopsticks[i].release()
                    self.chopstick_holders[i] = '     '

In [5]:
def main_semaphore():
    n = 5 
    m = 9
    dining_philosophers_w_semaphores = DiningPhilosophers_w_Semaphores(n, m)
    philosophers = [] 
    for _ in range(n):
        philosopher = threading.Thread(target=dining_philosophers_w_semaphores.philosopher, args=[_])  # the input is the id of each philosopher considered
        philosopher.start()
        philosophers.append(philosopher)

    ### this part could be skipped - we're using it for debugging and presentational purposes ###
    while sum(dining_philosophers_w_semaphores.meals) > 0:
        print("=" * (n*5))
        print("".join(map(str, dining_philosophers_w_semaphores.status)), 
                " : ", 
                str(dining_philosophers_w_semaphores.status.count('  E  '))
            )
        print("". join(map(str, dining_philosophers_w_semaphores.chopstick_holders)))
        print(dining_philosophers_w_semaphores.meals)
        time.sleep(0.1)
    ### this part could be skipped - we're using it for debugging and presentational purposes ###

    for _ in range(n):
        philosopher.join()


In [ ]:
# Why Deadlocks Are Avoided:
# No Circular Wait:
# If a philosopher fails to acquire their right chopstick within 1 second, they release their left chopstick, breaking potential dependency chains.
# Timeout Breaks Hold-and-Wait:
# Philosophers can't indefinitely hold one chopstick while waiting for another.
# No Full Symmetry:
# The random timeout duration creates staggered attempts, reducing simultaneous contention

# However, starvation is still possible for at least one philosopher: 
# 1. Unfair Retry Scheduling
# Problem: A philosopher might repeatedly fail to acquire both chopsticks due to bad timing.
# Example:
# Philosopher 2 always finds chopstick 3 taken by Philosopher 3, who consistently acquires it first.
# 2. No Prioritization
# First-Come-First-Serve: Active philosophers near hungry ones might "steal" chopsticks first.
# Resource Dominance: Adjacent philosophers could form an "alliance" unintentionally blocking a middle philosopher.

In [ ]:
main_semaphore()

  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  T    _    T    T    _    :  0
      /              /   
[9, 9, 9, 9, 9]
  T    _    T    T    _    :  0
      /              /   
[9, 9, 9, 9, 9]
  _    _    T    _    _    :  0
 /    /         /    /   
[9, 9, 9, 9, 9]
  _    _    _    _    _    :  0
 /    /    /    /    /   
[9, 9, 9, 9, 9]
  _    _    _    _    _    :  0
 /    /    /    /    /   
[9, 9, 9, 9, 9]
  _    _    _    _    _    :  0
 /    /    /    /    /   
[9, 9, 9, 9, 9]
  _    _    _    _    _    :  0
 /    /    /    /    /   
[9, 9, 9, 9, 9]
  _    _    _    _    _    :  0
 /    /    /    /    /   
[9, 9, 9, 9, 9]
  _    _    _    _    _    :  0
 /    /    /    /    /   
[9, 9, 9, 9, 9]
  _    _    _    _    _    :  0
 /    

## ✅ Solution 3: Using Conditional Variables

In [7]:
class DiningPhilosophers_w_ConditionVariables:

    def __init__(self, total_phils_no=5, meal_size=7):
        self.total_phils_no = total_phils_no  # same as before - each philosopher represents a "process" that entails "eating" `meal_size` times in order to conclude
        self.meals = [meal_size for _ in range(self.total_phils_no)]  # same as before - "eating" is a job to be done
        
        # The handling of the exclusive resources now changes:
        self.chopsticks = [False] * self.total_phils_no  # Boolean list indicating if a chopstick is in use - at the beginning, all philosophers are 'thinking', so no chopstick is in use
        self.lock = threading.Lock()  # A condition variable is always used together with a lock.
        # Here we have only a single lock for accessing shared resources - now, the resources to which we require exclusive access is just the 'chopsticks' list.
        self.condition = threading.Condition(self.lock)  # conditional variable for synchronization

        self.status = ['  T  ' for _ in range(self.total_phils_no)]  # Thinking status
        self.chopstick_holders = ['     ' for _ in range(self.total_phils_no)]  # Who holds chopsticks?


    # condition to be used in combination with the conditional variable !!!
    def can_eat(self, i):
        """ Check if a philosopher can eat (i.e., both his left AND right chopsticks are free) - chopstick indexes move clockwise"""
        left = i
        right = (i+1) % self.total_phils_no
        return not (self.chopsticks[right] and self.chopsticks[left])  # if both of them are False, he may eat (can_eat=True)
    

    def philosopher(self, i):
        while self.meals[i] > 0:  # the philosopher has not finished eating so he's in the lookout for chopsticks to do so
            self.status[i] = '  T  ' # starting from a thinking phase
            time.sleep(random.random())

            self.status[i] = '  _  '  # on the lookout to acquire chopsticks

            with self.condition:
                while not self.can_eat(i):  # wait until both chopsticks become available 
                    self.condition.wait()  # wait until it gets notified to see whether then both chopsticks have become available - until then, he may not touch anything
                
                # no they may eat
                # acquiring chopsticks
                left = i
                right = (i+1) % self.total_phils_no
                # here we're working with the exclusive resource so we ought to use the lock
                # self.lock.acquire()
                self.chopsticks[left] = True
                self.chopsticks[right] = True

                self.chopstick_holders[i] = ' / \\ '
                self.status[i] = '  E  '  # eating
                time.sleep(random.random())  # representing the actual case of eating, which takes up some seconds

                # Eating completed
                self.meals[i] -= 1

                # releasing chopsticks after eating
                self.chopsticks[left] = False
                self.chopsticks[right] = False
                # done working with the exclusive resource
                # self.lock.release()

                # Notify all waiting philosophers that at least one chopstick has now become available !
                self.condition.notify_all()


In [8]:
def main_cv():
    n = 5 
    m = 9

    dining_philosophers_w_cv = DiningPhilosophers_w_ConditionVariables(n, m)
    philosophers = []

    for _ in range(n):
        philosopher = threading.Thread(target=dining_philosophers_w_cv.philosopher, args=[_])
        philosophers.append(philosopher)
        philosopher.start()

    ### this part could be skipped - we're using it for debugging and presentational purposes ###
    while sum(dining_philosophers_w_cv.meals) > 0:
        print("=" * (n * 5))
        print("".join(map(str, dining_philosophers_w_cv.status)), 
                " : ", 
                str(dining_philosophers_w_cv.status.count('  E  '))
            )
        print("".join(map(str, dining_philosophers_w_cv.chopstick_holders)))
        print(dining_philosophers_w_cv.meals)
        time.sleep(0.1)
    ### this part could be skipped - we're using it for debugging and presentational purposes ###


    for philosopher in philosophers:
        philosopher.join()


In [ ]:
# Why Deadlocks Are Prevented:
# No Partial Acquisition:
# Philosophers only proceed when both chopsticks are available, eliminating the "hold one, wait for another" scenario.
# Centralized Control:
# The shared lock ensures only one philosopher can check/claim chopsticks at a time.
# No Circular Wait:
# The can_eat() check prevents philosophers from entering a state where each holds one chopstick.


# Why Starvation Is Avoided:
# Global Notifications:
# Every chopstick release triggers a wakeup for all waiting philosophers.
# Rechecking Conditions:
# Woken philosophers re-evaluate can_eat(), giving everyone a fair chance to eat.
# FIFO Queue:
# Condition variables internally manage a waiting queue, ensuring oldest requests get priority.


In [9]:
main_cv()

  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  T    T    T    T    T    :  0
                         
[9, 9, 9, 9, 9]
  E    T    T    T    T    :  1
 / \                     
[9, 9, 9, 9, 9]
  E    T    T    _    _    :  1
 / \                     
[9, 9, 9, 9, 9]
  E    T    _    _    _    :  1
 / \                     
[9, 9, 9, 9, 9]
  E    T    _    _    _    :  1
 / \                     
[9, 9, 9, 9, 9]
  E    T    _    _    _    :  1
 / \                     
[9, 9, 9, 9, 9]
  E    T    _    _    _    :  1
 / \                     
[9, 9, 9, 9, 9]
  E    T    _    _    _    :  1
 / \                     
[9, 9, 9, 9, 9]
  T    T    _    E    _    :  1
 / \            / \      
[8, 9, 9, 9, 9]
  T    _    _    E    _    :  1
 / \            / \      
[8, 9, 9, 9, 9]
  T    _    _    E    _    :  1
 / \            / \      
[8, 9, 9, 9, 9]
  T    _    _    E    _    :  1
 / \            / \      
[8, 9, 9, 9, 9]
  T    _    _    E    _    :  1
 / \  